In [27]:
from helpers.Data import load_queries
q = load_queries("./data/raw/traindev", total_window_tokens=44)


100%|██████████| 691/691 [00:00<00:00, 996.99it/s] 


In [28]:
import numpy as np
ls = []
tokens_len = []

for i, r in enumerate(q):
    ls.append(len(r[2]))
    tokens_len.append( len(r[2].split(" ")) )

percentile = np.percentile(np.array(ls), 97)
percentile

np.float64(266.97000000000025)

In [18]:
import numpy as np
np.percentile(np.array(ls), 97)

np.float64(278.0)

In [11]:
C0000039

array(['cardiomyopathy', 'MESH:D009202',
       'Emery-Dreifuss muscular dystrophy (EDMD) is an X-linked recessive muscular dystrophy characterized by early contractures of the elbows, Achilles tendons and spine, slowly progressive muscle wasting and weakness, and [MS] cardiomyopathy [ME] associated with cardiac conduction defects',
       './data/raw/traindev/10323252.txt', '339', '353'], dtype='<U609')

In [13]:
with open("./data/raw/traindev/10323252.txt", "r", encoding="utf-8") as f:
    l = f.read()

In [14]:
l

'Changes at P183 of emerin weaken its protein-protein interactions resulting in X-linked Emery-Dreifuss muscular dystrophy.\nEmery-Dreifuss muscular dystrophy (EDMD) is an X-linked recessive muscular dystrophy characterized by early contractures of the elbows, Achilles tendons and spine, slowly progressive muscle wasting and weakness, and cardiomyopathy associated with cardiac conduction defects. The emerin gene has been mapped to Xq28 and encodes a 34-kDa serine-rich protein, emerin, which has been localized to the nuclear envelope in a wide variety of tissues, including skeletal and cardiac muscle. Mutations spanning the emerin gene have been identified in patients with EDMD. We present here the effect, on emerin protein expression, of two missense mutations identified in unrelated EDMD patients. These alterations predict the replacement of a proline residue at position 183 with either a histidine or a threonine. Biochemical analysis has demonstrated that the mobility and expression 

In [51]:
import re
s = "Python is cool. Hello I AM my friends from hollywood that are working so hard. how. are you all ?"
mention = "my friends"
window_all = 7
mention_start = s.find(mention)
mention_end = mention_start + len(mention)



Hello I AM my friends from hollywood that are working so hard.
['Hello', 'I', 'AM', 'my', 'friends', 'from', 'hollywood', 'that', 'are', 'working', 'so', 'hard', '.']


'Hello I AM [MS] my friends [ME] from hollywood'

In [20]:
mention_token_start = next(i for i, (a, b) in enumerate(tokens_offsets) if a <= mention_start < b)
mention_token_end = next(i for i, (a, b) in enumerate(tokens_offsets) if b >= mention_end) + 1

tokens[mention_token_start: mention_token_end]

['my', 'friends']

In [23]:
right_space = len(tokens) - mention_token_end
tokens[right_space]

'you'

In [31]:
left_dot_pos = s.rfind('.', 0, mention_start)
left_start = left_dot_pos + 1 if left_dot_pos > -1 else 0
left_part = s[left_start:mention_start]

right_dot_pos = s.find('.', mention_end)
right_end = right_dot_pos if right_dot_pos > -1 else len(s)
right_part = s[mention_start+len(mention):right_end]

annotated = f"{left_part.strip()} [MS] {s[mention_start:mention_end].strip()} [ME] {right_part.strip()}"
annotated = re.sub(r'\s+', ' ', annotated).strip()
annotated

'The [MS] adenomatous polyposis coli (APC) tumour [ME] -suppressor protein controls the Wnt signalling pathway by forming a complex with glycogen synthase kinase 3beta (GSK-3beta), axin/conductin and betacatenin'

In [73]:
import numpy as np 
from collections import defaultdict


dictionary_cuis = np.array(["c_1", "c_2", "c_3", "c_1", "c_4", "c_5", "c_2", "c_3", "c_4", "c_1", "c_4", "c_2", "c_2"])
queries_cuis = np.array(["c_1", "c_2", "c_5"])
num_queries = 2
topk = 5
new_cands = np.array([
    [1,5,0,3,11], # 0,3 are true
    [0,11,6,1,12], # 1 is true
    [0,1,10,7,2], # 5 is true
])

previous_epoch_candidates = np.array([
    [0,3, 9, 7, 8], #7 or 8 should be injected
    [1,6,11,12, 3], # 3 should be injected
    [5,0,1,2,3] #not the 5
])

dictionary_cui_to_idx =  defaultdict(list)
for i, d in enumerate(dictionary_cuis):
    dictionary_cui_to_idx[d].append(i)
dictionary_cui_to_idx

defaultdict(list,
            {np.str_('c_1'): [0, 3, 9],
             np.str_('c_2'): [1, 6, 11, 12],
             np.str_('c_3'): [2, 7],
             np.str_('c_4'): [4, 8, 10],
             np.str_('c_5'): [5]})

In [104]:
import torch
new_cands_1 = torch.tensor(new_cands).clone()
new_cands_2 = torch.tensor(new_cands).clone()

In [111]:

query_idx = 1
hard_positives_num = 2
hard_negatives_num = 1
query_cui = queries_cuis[query_idx]
current_query_candidates_idxs = new_cands[query_idx].tolist()
current_candidates_cuis = dictionary_cuis[current_query_candidates_idxs]


positive_positions = np.where(current_candidates_cuis == query_cui)[0]
candidates_idxs_available = list(set(range(topk))  - set(positive_positions)  )
positive_candidates_indexes = dictionary_cui_to_idx.get(query_cui, [])
available_positives = list(set(positive_candidates_indexes) - set(current_query_candidates_idxs))


if available_positives:
    # how many positives we will inject, in case available are less than the one in config
    positive_n = min(hard_positives_num, len(available_positives))
    #  random positive candidates, to choose from available positives (index of dictionary_cui)
    positive_candidates = np.array(available_positives[:positive_n])
    # random indexes in candidate list to be replaced
    candidates_idxs_to_be_replaced = candidates_idxs_available[:positive_n]
    new_cands_2[query_idx, candidates_idxs_to_be_replaced] = torch.from_numpy(positive_candidates)



new_cands_2[query_idx]

tensor([ 3, 11,  6,  1, 12])

In [112]:
import torch

query_cui = queries_cuis[query_idx]
current_idxs = np.array(new_cands[query_idx])
current_cuis = dictionary_cuis[current_idxs]

negative_mask = (current_cuis != query_cui)
available_positions = np.flatnonzero(negative_mask)

pos_dict_idxs = dictionary_cui_to_idx.get(query_cui, [])
available_pos_dict = np.setdiff1d(pos_dict_idxs, current_idxs, assume_unique=False)

pos_n = min(hard_positives_num,
            len(available_pos_dict),
            len(available_positions))
chosen_pos_dict = available_pos_dict[:pos_n]
chosen_slots = available_positions[:pos_n]
new_cands_1[query_idx, chosen_slots] = torch.from_numpy(chosen_pos_dict)
available_positions = np.setdiff1d(available_positions, chosen_slots, assume_unique=False)


new_cands_1[query_idx]

tensor([ 3, 11,  6,  1, 12])

In [74]:
import torch

query_idx = 1
hard_positives_num = 2
hard_negatives_num = 1
query_cui = queries_cuis[query_idx]
current_query_candidates_idxs = new_cands[query_idx].tolist()
current_candidates_cuis = dictionary_cuis[current_query_candidates_idxs]
positive_positions = np.where(current_candidates_cuis == query_cui)[0]

candidates_idxs_to_be_replaced = np.array([])
candidates_idxs_available = list(set(range(topk))  - set(positive_positions)  )

print(f"current candidates are: {new_cands[query_idx]}")

positive_candidates_indexes = dictionary_cui_to_idx.get(query_cui, [])
if len(positive_candidates_indexes) > 0:
    available_positives = list(set(positive_candidates_indexes) - set(current_query_candidates_idxs))
    if available_positives:
        positive_n = min(hard_positives_num, len(available_positives))
        positive_candidates = np.random.choice(available_positives, size=positive_n, replace=False)

        candidates_available_idxs = candidates_idxs_available
        candidates_idxs_to_be_replaced = np.random.choice(candidates_idxs_available, size=positive_n, replace=False)
        new_cands[query_idx, candidates_idxs_to_be_replaced] = torch.from_numpy(positive_candidates)

print(f"after injecting {hard_positives_num} positive are: {new_cands[query_idx]}")
candidates_idxs_available = list(set(candidates_idxs_available) - set(candidates_idxs_to_be_replaced))
prev_cands_idxs = previous_epoch_candidates[query_idx]
prev_dictionary_cuis = dictionary_cuis[prev_cands_idxs]
neg_mask = prev_dictionary_cuis != query_cui
hard_negative_indexes = prev_cands_idxs[neg_mask]

if len(hard_negative_indexes) > 0:
    negatives_n = min(hard_negatives_num, len(hard_negative_indexes))
    hard_negative_candidates = np.random.choice(hard_negative_indexes, size=negatives_n, replace=False)
    # candidates_to_replace_positive
    candidates_idxs_to_be_replaced = np.random.choice(candidates_idxs_available, size=negatives_n, replace=False)

    new_cands[query_idx, candidates_idxs_to_be_replaced] = torch.from_numpy(hard_negative_candidates)

print(f"after injecting {hard_negatives_num} negatives are: {new_cands[query_idx]}")

new_cands[query_idx]

current candidates are: [ 0 11  6  1 12]
after injecting 2 positive are: [ 0 11  6  1 12]
after injecting 1 negatives are: [ 3 11  6  1 12]


array([ 3, 11,  6,  1, 12])